In [2]:
import pandas as pd
from ggplot import *

project_id = "the-d-team-164512"

In [13]:
query ="""
    with p as (
    select  pat_state, count(distinct patient_id) as pat_dup_cnt
    from dthon.duplicates_enriched
    group by pat_state),
    p_l as (
    select count(distinct patient_id) as pat_cnt, LPAD(LPAD(CAST(postcode as string),4,'0'), 1) as pat_state2
    from dthon.patients_lookup
    group by pat_state2),
    s as (
    select count(distinct store_id) as str_dup_cnt, str_state
    from dthon.duplicates_enriched
    group by str_state),
    s_l as (
    select count(distinct store_id) as str_cnt, LPAD(LPAD(CAST(postcode as string),4,'0'), 1) as str_state2
    from dthon.stores_lookup
    group by str_state2)
    select p.*, p_l.pat_cnt, s.str_dup_cnt, s_l.str_cnt
    from p
    left join p_l
    on p.pat_state = p_l.pat_state2
    left join s
    on p.pat_state = s.str_state
    left join s_l
    on p.pat_state = s_l.str_state2
  """


First we would like to check whether all tasmanian patients have duplicates and all tasmanian stores also.

In [11]:
dup_pat_str = pd.read_gbq(query, project_id,dialect='standard')
dup_pat_str

Requesting query... ok.
Query running...
Query done.
Processed: 49.6 Mb

Retrieving results...
  Got page: 1; 100% done. Elapsed 10.96 s.
Got 7 rows.

Total time taken 10.97 s.
Finished at 2017-04-21 00:27:48.


,pat_state,pat_dup_cnt,pat_cnt,str_dup_cnt,str_cnt
0,3,602,180800,496,774
1,2,1588,201586,694,1085
2,5,155,36478,130,192
3,4,287,98990,358,487
4,6,82,20972,86,180
5,7,19078,19078,72,73
6,0,15,448,24,30


We see that one store doesn't generate duplicates (tasmania - 7). Which one it is and whether it has any transactions

In [14]:
query = """
select distinct store_id as str_cnt
from dthon.stores_lookup
where LPAD(LPAD(CAST(postcode as string),4,'0'), 1) = '7'
and store_id not in (select distinct store_id as str_dup_cnt
from dthon.duplicates_enriched
where str_state = '7')
"""
store_no_dup = pd.read_gbq(query, project_id,dialect='standard')
store_no_dup  

Requesting query... ok.
Query running...
Query done.
Processed: 20.5 Mb

Retrieving results...
Got 1 rows.

Total time taken 4.64 s.
Finished at 2017-04-21 00:35:49.


,str_cnt
0,1307


In [15]:
query = """
select Patient_ID
from dthon.transactions
where Store_ID = 1307
"""
store_1307 = pd.read_gbq(query, project_id,dialect='standard')
store_1307 

Requesting query... ok.
Query running...
Query done.
Processed: 907.1 Mb

Retrieving results...
Got 0 rows.

Total time taken 3.88 s.
Finished at 2017-04-21 00:35:57.


,Patient_ID


That explains why there were no duplicates. On the other hand side:
* why is there a store without transactions
* how many are there

Before  that, let's check if all transactions from all tasmanian stores are duplicated.

In [16]:
query = """
with tr as (
select count(1) as cnt_all, LPAD(LPAD(CAST(postcode as string),4,'0'), 1) as state
from dthon.transactions tr
Left join dthon.stores_lookup as sl
on sl.Store_id = tr.Store_ID
group by state
), dup as (
select str_state, sum(cnt) as cnt_dup
from dthon.duplicates_enriched 
group by str_state
)
select dup.*, tr.cnt_all
from dup
left join tr
on tr.state = dup.str_state
"""
dup_tr_region = pd.read_gbq(query, project_id,dialect='standard')
dup_tr_region

Requesting query... ok.
Query running...
Query done.
Processed: 474.1 Mb

Retrieving results...
Got 8 rows.

Total time taken 6.07 s.
Finished at 2017-04-21 00:48:58.


,str_state,cnt_dup,cnt_all
0,7,3785978,3835232.0
1,4,14263,9489847.0
2,0,2066,68213.0
3,3,17902,19325433.0
4,6,2181,1902308.0
5,None,60906,NaN
6,2,28368,21234352.0
7,5,4769,3532609.0


There are some stores with no state assigned:
* wrong query to create duplicates dataset?
* stores without postcode in lookup?
* stores in transactions without foreign code in lookup file?